In [110]:
import numpy as np
import matplotlib.pyplot as plt

datadir = '../data/kddcup99/'

# anomaly
x2 = np.load(datadir + 'anomaly_std.npy').squeeze()
y2 = np.load(datadir + 'anomaly_mu_loss.npy').squeeze()
# normal
x1 = np.load(datadir + 'normal_std.npy').squeeze()
y1 = np.load(datadir + 'normal_mu_loss.npy').squeeze()

print(x2.shape, y2.shape, x1.shape, y1.shape)

(48880, 5) (48880, 121) (198131, 5) (198131, 121)


In [111]:
x2sum = np.sum(x2, axis = 1).squeeze()
y2sum = np.sum(y2, axis = 1).squeeze()
x1sum = np.sum(x1, axis = 1).squeeze()
y1sum = np.sum(y1, axis = 1).squeeze()

# fig = plt.figure()
# ax = plt.subplot()
# ax.scatter(x1sum, y1sum,s=1, c='green', alpha=0.6)
# ax.scatter(x2sum, y2sum,s=1, c='red', alpha=0.6) 

threshold = 25
print(len(y2sum[(y2sum < 10)]), len((y1sum[y1sum < 10])))
print(len(y2sum[y2sum > threshold]), len(y1sum[y1sum > threshold]))

print(x2sum.mean(), y2sum.mean(), x1sum.mean(), y1sum.mean())
print(x2sum.shape, y2sum.shape, x1sum.shape, y1sum.shape)



recall = len(y2sum[y2sum > threshold]) / 48880
presicion = len(y2sum[y2sum > threshold]) / (len(y2sum[y2sum > threshold])+len(y1sum[y1sum > threshold]))
f1 = 2*recall*presicion / (recall + presicion)
print(f1)

0 140009
48722 7596
0.006105543591652741 188.82746833591997 0.12505401728267138 88.49631214658446
(48880,) (48880,) (198131,) (198131,)
0.9262913743607293


In [196]:
np.concatenate([y1sum, y2sum])

percentage = [97.5, 95, 92.5]
anomaly_percentage = [95.7, 95.6, 95.5]

upper = np.percentile(np.concatenate([y1sum, y2sum]), percentage[0])
lower = np.percentile(np.concatenate([y1sum, y2sum]), 50)

anomaly_upper = np.percentile(y2sum, anomaly_percentage[2])
print('upper:', upper, 'lower:', lower, 'anomaly_upper:', anomaly_upper)

upper: 300.5014869045385 lower: 8.27565501868844 anomaly_upper: 266.0199883118474


In [197]:
# first is normal, secord is anomaly

# kdd99 
# upper = 200 #102.5 #10000000 #102.5 #58.5  #46    #20.22
# lower = 10 #0.1
# upper1 = 20.15  #7.6      #16    #20.06 #20.15 #nan 

# # letter
# upper = 41.6  #31.3 #33.8 #37 #41.6
# lower = 17.5

# # thyroid
# upper = 2 #0.4  #0.57 #0.95 #2
# lower = 0.04

# y1 = y1sum
# y2 = y2sum
# x1 = x1sum
# x2 = x2sum
print(y1.shape, y2.shape, x1.shape, x2.shape)

anomalyInstance = np.concatenate((y1[np.where(y1sum > upper)[0]], x1[np.where(y1sum > upper)[0]]), axis=1)
tmp = np.concatenate((y2[np.where(y2sum > upper)[0]], x2[np.where(y2sum > upper)[0]]), axis=1)
# anomalyInstance = np.concatenate((anomalyInstance, tmp), axis=0)

anomalyLableInstance = np.concatenate((y2[np.where(y2sum > anomaly_upper)[0]], x2[np.where(y2sum > anomaly_upper)[0]]), axis=1)
anomalyInstance = np.concatenate((anomalyInstance, tmp, anomalyLableInstance), axis=0)

print('anomaly:',anomalyInstance.shape)


normalInstance = []
normalInstance = np.concatenate((y1[np.where(y1sum < lower)[0]], x1[np.where(y1sum < lower)[0]]), axis=1)
tmp = np.concatenate((y2[np.where(y2sum < lower)[0]], x2[np.where(y2sum < lower)[0]]), axis=1)

normalInstance = np.concatenate((normalInstance, tmp), axis=0)
print('normal:', normalInstance.shape)
instances = np.concatenate((anomalyInstance, normalInstance), axis=0)
print('total instance:', instances.shape)
labels = [1 for i in range(len(anomalyInstance))]
labels.extend([0 for i in range(len(normalInstance))])
labels = np.array(labels)
print('total label:', labels.shape)

for i in range(1, len(normalInstance) // len(anomalyInstance)):
    instances = np.concatenate((instances, anomalyInstance), axis=0)
    labels = np.concatenate((labels, [1 for j in range(len(anomalyInstance))]), axis=0)
print(len(instances), len(labels))
np.save(datadir + 'training_data.npy', instances)
np.save(datadir + 'traning_label.npy', labels)

anomalyInstance = np.concatenate((y2, x2), axis=1)
normalInstance = np.concatenate((y1, x1), axis=1)
instances = np.concatenate((anomalyInstance, normalInstance), axis=0)

labels = [1 for i in range(len(anomalyInstance))]
labels.extend([0 for i in range(len(normalInstance))])
print(len(anomalyInstance))
labels = np.array(labels)
print(instances.shape)

np.save(datadir + 'testing_data.npy', instances) 
np.save(datadir + 'testing_label.npy', labels)

(198131, 121) (48880, 121) (198131, 5) (48880, 5)
anomaly: (8376, 126)
normal: (123411, 126)
total instance: (131787, 126)
total label: (131787,)
240675 240675
48880
(247011, 126)
